# Notebook 4 : JSON et API

In [ ]:
import json
import random

from io import StringIO # Pour éviter les avertissements de read_json

import pandas as pd
import requests

## Format JSON

Nous considérons deux jeux de données artificiels pour illustrer des limites du format JSON à garder à l'esprit en pratique.

In [ ]:
nombres = pd.DataFrame({"Nombre": [random.random() for _ in range(5)]})
nananinf = pd.DataFrame({"Valeur": [3.14, pd.NA, float("nan"), float("inf")]})

1. Convertir `nombres` au format JSON avec la méthode `to_json` et stocker le résultat dans une variable `nombres_json`.

In [ ]:
nombres_json = nombres.to_json()
nombres_json

2. Importer `nombres_json` avec la fonction `read_json` de Pandas dans un dataframe `nombres_bis`. Comparer les objets `nombres` et `nombres_bis`.

In [ ]:
nombres_bis = pd.read_json(StringIO(nombres_json))

# Le stockage de flottants dans un format texte comme le JSON induit une perte de précision
(nombres - nombres_bis).abs().max()

3. Lire la documentation de `to_json` pour connaître l'option permettant de gérer (mais pas de résoudre) le problème précédent.

In [ ]:
# L'option double_precision permet de définir la précision des flottants exportés

def affiche_precision(df):
    print(f"Précision : {(nombres - df).abs().max().values}")

for precision in (5, 10, 15):
    print(f"double_precision = {precision}")
    affiche_precision(
        pd.read_json(
            StringIO(nombres.to_json(double_precision=precision))
        )
    )
    print()

4. Convertir `nananinf` au format JSON avec la méthode `to_json` et stocker le résultat dans une variable `nananinf_json`. Que sont devenus `NA`, `NaN` et `inf` ?

In [ ]:
nananinf_json = nananinf.to_json()

# NA (donnée manquante), nan (Not a Number) et inf (infini) sont devenus null
nananinf_json

5. Importer `nananinf_json` avec la fonction `read_json` de Pandas dans un dataframe `nananinf_bis`. Comparer les objets `nananinf` et `nananinf_bis`.

In [ ]:
nananinf_bis = pd.read_json(StringIO(nananinf_json))

# NA, nan et inf sont devenus des NaN de Pandas
nananinf_bis

6. Reprendre les questions 4 et 5 sur l'objet `[float("nan"), float("inf")]` avec les fonctions `dumps` et `loads`. Quelle est la différence ? Lire la documentation de `dumps` pour comprendre l'option `allow_nan`.

In [ ]:
# Liste des valeurs de nananinf
naninf_list = [float("nan"), float("inf")]
naninf_list

In [ ]:
# Conversion en JSON
naninf_list_json = json.dumps(naninf_list)
naninf_list_json

In [ ]:
# Retour en Python depuis le JSON
# Les valeurs nan et inf sont préservées
json.loads(naninf_list_json)

In [ ]:
# La gestion de nan et inf en JSON n'est pas bien définie comme nous l'avons vu dans les questions précédentes.
# L'option allow_nan permet de bloquer ces conversions incertaines en levant une exception ValueError.

# json.dumps(naninf_list, allow_nan=False) # ValueError

## Iris

Nous reprenons ici le jeu de données des [Iris de Fisher](https://fr.wikipedia.org/wiki/Iris_de_Fisher) pour étudier les différentes façons d'exporter un dataframe au format JSON.

1. Charger le jeu de données dans un dataframe `iris` à partir du fichier `iris.csv`.

In [ ]:
iris = pd.read_csv("data/iris.csv")
iris

2. Comparer les résultats obtenus en exportant `iris` au format JSON avec `to_json` et :
- `orient="columns"`,
- `orient="index"`,
- `orient="records"`.

In [ ]:
# Choix par défaut
iris.to_json(orient="columns")

In [ ]:
# Objet unique
iris.to_json(orient="index")

In [ ]:
# Liste d'objets
iris.to_json(orient="records")

3. Exporter `iris` dans un fichier `iris.json` au format NDJSON. Ouvrir ce fichier dans un éditeur de texte pour vérifier que chaque ligne contient un document.

In [ ]:
iris.to_json("iris.json", orient="records", lines=True)

4. Importer le fichier `iris.json` au format NDJSON dans un dataframe `iris2`.

In [ ]:
iris2 = pd.read_json("iris.json", lines=True)
iris2

## Star Wars API

Le projet SWAPI (*Star Wars API*) est une source de données sur l'univers de Star Wars. L'API fournit plusieurs jeux de données concernant les planètes, les vaisseaux, les véhicules, les personnages, les films et les espèces de la saga venue d'une galaxie très, très lointaine.

1. Utiliser la fonction Pandas `read_json` pour importer les données sur les planètes disponibles au format JSON à l'adresse [https://swapi-node.vercel.app/api/planets](https://swapi-node.vercel.app/api/planets) dans un dataframe. Est-ce que le résultat est facilement exploitable sous cette forme ?

In [ ]:
swapi_url = "https://swapi-node.vercel.app"

planets_url = swapi_url + "/api/planets"
planets_test = pd.read_json(planets_url)
planets_test

2. Utiliser la fonction `get` du module `requests` pour récupérer les mêmes données que dans la question précédente et vérifier le code HTTP obtenu.

In [ ]:
r = requests.get(planets_url)

if r.status_code == 200:
    print("Données récupérées !")
else:
    print(f"Erreur {r.status_code}")

3. Comprendre les éléments de la réponse obtenue à la question précédente. En particulier, combien y a-t-il de planètes dans `results` et à quoi correspond `next` ?

In [ ]:
# Objet de la réponse
obj = r.json()
obj

In [ ]:
# Nombre total de planètes dans la base de données
obj["count"]

In [ ]:
# Nombre de planètes récupérées par notre requête
len(obj["results"])

In [ ]:
# Données de la première planète
obj["results"][0]["fields"]

In [ ]:
# URL de la requête pour obtenir les planètes suivantes
obj["next"]

4. Écrire une boucle pour récupérer les informations de toutes les planètes disponibles dans l'API et stocker le résultat dans un dataframe `planets`.

In [ ]:
planets_data = []
next_url = planets_url
while next_url is not None:
    print(f"Téléchargement des données {next_url}")
    r = requests.get(next_url)
    if r.status_code != 200:
        print(f"Erreur {r.status_code}")
        break # Stop en cas d'erreur
    r_obj = r.json()
    planets_data.extend(
        [
            result["fields"]
            for result in r_obj["results"]
        ]
    )
    next_url = (
        None if r_obj["next"] is None
        else swapi_url + r_obj["next"]
    )

planets = pd.DataFrame(planets_data)
planets

5. Exporter le dataframe obtenu à la question précédente dans un fichier `planets.json` au format NDJSON.

In [ ]:
planets.to_json("planets.json", orient="records", lines=True)